In [1]:
try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
    IS_COLAB = True
except Exception:
    IS_COLAB = False

if IS_COLAB:
    !git clone https://github.com/Lenkus1/HuubS_ML_Opdracht.git
    %cd HuubS_ML_Opdracht/template
    #%pip install loguru
    %pip install -q -U keras-tuner

In [2]:
%load_ext autoreload
%autoreload 2

#%pip install keras-tuner


from pathlib import Path
import os
import sys
sys.path.insert(0, "..")
sys.path

import numpy as np
import seaborn as sns
sns.set_theme()

import pandas as pd

import tensorflow as tf
from tensorflow import keras

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

from keras.callbacks import ReduceLROnPlateau
from tensorflow.keras import layers
from tensorflow.keras import regularizers
import keras_tuner as kt

%load_ext autoreload
%autoreload 2

#%pip install keras-tuner


from pathlib import Path
import os
import sys
sys.path.insert(0, "..")
sys.path

import numpy as np
import seaborn as sns
sns.set_theme()

import pandas as pd

import tensorflow as tf
from tensorflow import keras

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

from keras.callbacks import ReduceLROnPlateau
from tensorflow.keras import layers
from tensorflow.keras import regularizers
import keras_tuner as kt


import random as python_random

# The below is necessary for starting Numpy generated random numbers
# in a well-defined initial state.
np.random.seed(42)

# The below is necessary for starting core Python generated random numbers
# in a well-defined state.
python_random.seed(42)

# The below set_seed() will make random number generation
# in the TensorFlow backend have a well-defined initial state.
# For further details, see:
# https://www.tensorflow.org/api_docs/python/tf/random/set_seed
tf.random.set_seed(42)





#data_set = "/content/HuubS_ML_Opdracht/template/src/data/"

#from HuubS_ML_Opdracht.template.src.data import make_dataset


#from HuubS_ML_Opdracht.template.src.visualization import visualize_simple





#data_set = "/content/HuubS_ML_Opdracht/template/src/data/"

#from HuubS_ML_Opdracht.template.src.data import make_dataset


#from HuubS_ML_Opdracht.template.src.visualization import visualize_simple





The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
datadir_processed = Path("../data/processed")
datadir_model = Path("../models")

if IS_COLAB:
    datadir_processed = "/content/HuubS_ML_Opdracht/template/data/processed"
    datadir_model = "/content/HuubS_ML_Opdracht/template/models"

In [4]:
from typing import Dict, List, Tuple

def cfm_heatmap(
    cfm: np.ndarray,
    figsize: tuple = (8, 8),
    scale: float = None,
    labels: List[str] = None,
    vmin: float = None,
    vmax: float = None,
) -> None:
    """
    figsize: tuple, default (8,8)
    scale: string. The direction over which the numbers are scaled.
        Either None, 'total', 'rowwise' or 'colwise'
    """

    if scale == "total":
        cfm_norm = cfm / np.sum(cfm)
    elif scale == "rowwise":
        cfm_norm = cfm / np.sum(cfm, axis=1, keepdims=True)
    elif scale == "colwise":
        cfm_norm = cfm / np.sum(cfm, axis=0, keepdims=True)
    else:
        cfm_norm = cfm
    plt.figure(figsize=figsize)
    if labels is not None:
        plot = sns.heatmap(
            cfm_norm,
            annot=cfm_norm,
            vmin=vmin,
            vmax=vmax,
            xticklabels=labels,
            yticklabels=labels,
        )
    else:
        plot = sns.heatmap(cfm_norm, annot=cfm_norm, vmin=vmin, vmax=vmax)
    plot.set(xlabel="Predicted", ylabel="Target")


In [5]:
data = []
labels = []

for f in sorted(os.listdir(datadir_processed)):
    folder = os.path.join(datadir_processed, f)
    if os.path.isdir(folder):
        #print(f"{f} is a target class")
        for i in sorted(os.listdir(folder)):
            image=tf.keras.preprocessing.image.load_img(folder+'/'+i, 
            target_size= (64,64))
            image=np.array(image)
            data.append(image)
            labels.append(f)

data = np.array(data)
#labels = np.array(labels)
labels = list(labels)
#labels_text = labels
encoder = LabelEncoder()
labels = encoder.fit_transform(labels)
#labels = to_categorical(labels)



X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2,
                                                random_state=42)

X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.125,
                                                random_state=42)




# Pre-processing
X_train = X_train.astype('float32') / 255.
X_test = X_test.astype('float32') / 255
X_valid = X_valid.astype('float32') / 255

print(X_train.shape)
print(X_test.shape)
print(X_valid.shape)


(12414, 64, 64, 3)
(3548, 64, 64, 3)
(1774, 64, 64, 3)


In [6]:
from kerastuner import HyperModel

INPUT_SHAPE = (64, 64, 3)
NUM_CLASSES = 10



class CNNHyperModel(HyperModel):
    def __init__(self, input_shape, num_classes):
        self.input_shape = input_shape
        self.num_classes = num_classes

    def build(self, hp):
        
        KS=hp.Choice('num_kernels',values=[3, 5, 7], default=5)
        KI='he_uniform'
        KR=regularizers.l2(hp.Float('l2', min_value=0.001, max_value=0.005, default=0.002, step=0.001))
      
        data_augmentation = tf.keras.Sequential([
        keras.layers.RandomFlip("horizontal"),
        keras.layers.RandomRotation(0.2),
        keras.layers.RandomTranslation(0.1, 0.1),
        keras.layers.RandomZoom(0.2),
        ])

        
        model = keras.Sequential()

        data_augmentation

  
        model.add(keras.layers.Conv2D(filters=hp.Choice('num_filters_1', values=[32, 64, 96, 128], default=32), kernel_size=KS,kernel_initializer=KI, kernel_regularizer=KR, input_shape=self.input_shape))
        model.add(keras.layers.Conv2D(filters=hp.Choice('num_filters_2', values=[32, 64, 96, 128], default=32), kernel_size=KS,kernel_regularizer=KR))
        model.add(keras.layers.LeakyReLU(alpha=0.2))
        model.add(keras.layers.MaxPooling2D(pool_size=2))
        model.add(keras.layers.BatchNormalization())
        model.add(keras.layers.Dropout(rate=hp.Float('dropout_1', min_value=0.0, max_value=0.5, default=0.25, step=0.05)))
       
        model.add(keras.layers.Conv2D(filters=hp.Choice('num_filters_3', values=[32, 64, 96, 128], default=64), kernel_size=KS,kernel_regularizer=KR))
        model.add(keras.layers.Conv2D(filters=hp.Choice('num_filters_4', values=[32, 64, 96, 128], default=64), kernel_size=KS,kernel_regularizer=KR))
        model.add(keras.layers.LeakyReLU(alpha=0.2))
        model.add(keras.layers.MaxPooling2D(pool_size=2))
        model.add(keras.layers.BatchNormalization())
        model.add(keras.layers.Dropout(rate=hp.Float('dropout_2', min_value=0.0, max_value=0.5, default=0.25, step=0.05)))

       
        model.add(keras.layers.Conv2D(filters=hp.Choice('num_filters_5', values=[32, 64, 96, 128], default=64), kernel_size=KS,kernel_regularizer=KR))
        model.add(keras.layers.LeakyReLU(alpha=0.2))
        model.add(keras.layers.BatchNormalization())

        model.add(keras.layers.Flatten())
        model.add(keras.layers.Dense(units=hp.Int('units_1', min_value=32, max_value=256, step=32, default=128), activation=hp.Choice('dense_activation', values=['relu', 'tanh', 'sigmoid'], default='relu'),kernel_regularizer=KR))
        model.add(keras.layers.Dropout(rate=hp.Float('dropout_3', min_value=0.0, max_value=0.5, default=0.25, step=0.05)))
        model.add(keras.layers.Dense(units=hp.Int('units_2', min_value=32, max_value=128, step=16, default=64), activation=hp.Choice('dense_activation', values=['relu', 'tanh', 'sigmoid'], default='relu'),kernel_regularizer=KR))
        model.add(keras.layers.Dropout(rate=hp.Float('dropout_4', min_value=0.0, max_value=0.5, default=0.25, step=0.05)))
        model.add(keras.layers.Dense(self.num_classes, activation='softmax'))

        
        model.compile(
            optimizer=keras.optimizers.Adam(
                hp.Float(
                    'learning_rate',
                    min_value=1e-4,
                    max_value=1e-2,
                    sampling='LOG',
                    default=1e-3
                )
            ),
            loss='sparse_categorical_crossentropy',
            metrics=['accuracy']
        )
        return model

hypermodel = CNNHyperModel(input_shape=INPUT_SHAPE, num_classes=NUM_CLASSES)

C:\Users\hjtfs\AppData\Local\Temp/ipykernel_420/752153518.py:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner import HyperModel


In [7]:
from kerastuner.tuners import Hyperband

tuner = kt.Hyperband(
    hypermodel,
    overwrite=True,
    objective='val_accuracy',
    max_epochs=20,
    executions_per_trial=1,
    hyperband_iterations=1,
    directory=datadir_model,
    project_name='galaxies'
    )


In [8]:
tuner.search_space_summary()

Search space summary
Default search space size: 15
num_kernels (Choice)
{'default': 5, 'conditions': [], 'values': [3, 5, 7], 'ordered': True}
l2 (Float)
{'default': 0.002, 'conditions': [], 'min_value': 0.001, 'max_value': 0.005, 'step': 0.001, 'sampling': None}
num_filters_1 (Choice)
{'default': 32, 'conditions': [], 'values': [32, 64, 96, 128], 'ordered': True}
num_filters_2 (Choice)
{'default': 32, 'conditions': [], 'values': [32, 64, 96, 128], 'ordered': True}
dropout_1 (Float)
{'default': 0.25, 'conditions': [], 'min_value': 0.0, 'max_value': 0.5, 'step': 0.05, 'sampling': None}
num_filters_3 (Choice)
{'default': 64, 'conditions': [], 'values': [32, 64, 96, 128], 'ordered': True}
num_filters_4 (Choice)
{'default': 64, 'conditions': [], 'values': [32, 64, 96, 128], 'ordered': True}
dropout_2 (Float)
{'default': 0.25, 'conditions': [], 'min_value': 0.0, 'max_value': 0.5, 'step': 0.05, 'sampling': None}
num_filters_5 (Choice)
{'default': 64, 'conditions': [], 'values': [32, 64, 96, 

In [9]:
tuner.search(X_train, y_train,
             validation_data=(X_valid, y_valid),
             callbacks=[tf.keras.callbacks.EarlyStopping('val_loss', patience=2)])

Trial 2 Complete [00h 13m 58s]
val_accuracy: 0.4763246774673462

Best val_accuracy So Far: 0.4763246774673462
Total elapsed time: 00h 22m 57s

Search: Running Trial #3

Hyperparameter    |Value             |Best Value So Far 
num_kernels       |5                 |7                 
l2                |0.004             |0.002             
num_filters_1     |128               |128               
num_filters_2     |64                |32                
dropout_1         |0.2               |0.3               
num_filters_3     |128               |128               
num_filters_4     |128               |64                
dropout_2         |0.1               |0.45              
num_filters_5     |128               |96                
units_1           |64                |192               
dense_activation  |sigmoid           |sigmoid           
dropout_3         |0.5               |0.15              
units_2           |32                |80                
dropout_4         |0.15          

In [ ]:
# Show a summary of the search
tuner.results_summary()

In [ ]:
best_hps=tuner.get_best_hyperparameters()[0]

In [ ]:
# Build the model with the optimal hyperparameters and train it on the data for 50 epochs
model = tuner.hypermodel.build(best_hps)
history = model.fit(X_train, y_train, epochs=20,validation_data=(X_valid, y_valid))

val_loss_per_epoch = history.history['val_loss']
best_epoch = val_loss_per_epoch.index(min(val_loss_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch,))

# Retrieve the best model.
best_model = tuner.get_best_models(num_models=1)[0]

In [ ]:
# Evaluate the best model.
loss, accuracy = best_model.evaluate(X_test, y_test)

In [ ]:
pd.DataFrame(history.history).plot(figsize=(15, 15))
plt.grid(True)
plt.gca().set_ylim(0, 2) # set the vertical range to [0-2]
plt.show()

In [ ]:
#y_test = np.argmax(y_test,axis=1)
predict = best_model.predict(X_test).argmax(axis=1)


In [ ]:
#encoder = LabelEncoder()
#y_test = encoder.fit_transform(y_test)
#predict = encoder.fit_transform(predict)

print(classification_report(y_test, predict))

In [ ]:
cfm = confusion_matrix(y_test, predict)
cfm_heatmap(cfm, figsize=(12,12), scale='rowwise', vmax= 0.8)




In [ ]:
best_model_fix = keras.models.Sequential([

keras.layers.Conv2D(filters=96, kernel_size=(3,3), input_shape=(INPUT_SHAPE)),
keras.layers.Conv2D(filters=64, kernel_size=(3,3)),
keras.layers.LeakyReLU(alpha=0.2)),
keras.layers.MaxPooling2D(pool_size=(2,2)),
keras.layers.BatchNormalization(),
keras.layers.Dropout(0.15)

keras.layers.Conv2D(filters=128, kernel_size=(3,3)),
keras.layers.Conv2D(filters=128, kernel_size=(3,3)),
keras.layers.LeakyReLU(alpha=0.2)),
keras.layers.MaxPooling2D(pool_size=(2,2)),
keras.layers.BatchNormalization(),
keras.layers.Dropout(0.20)

keras.layers.Conv2D(filters=96, kernel_size=(3,3),
keras.layers.LeakyReLU(alpha=0.2)),
keras.layers.BatchNormalization(),


keras.layers.Flatten(),
keras.layers.Dense(64, activation="relu"),
keras.layers.Dropout(0.25),
keras.layers.Dense(64, activation="relu"),
keras.layers.Dropout(0.5),
keras.layers.Dense(10, activation="softmax")
])

best_model_fix.summary()

In [ ]:

        best_model_fix.compile(
            optimizer=keras.optimizers.Adam(
                hp.Float(
                    'learning_rate',
                    min_value=1e-4,
                    max_value=1e-2,
                    sampling='LOG',
                    default=1e-3
                )
            ),
            loss='sparse_categorical_crossentropy',
            metrics=['accuracy']
        )

reduceLR = ReduceLROnPlateau(monitor='val_accuracy', 
factor=.001, 
patience=2, 
min_delta=0.01, 
mode="auto")

In [ ]:
hist_best_model_fix = best_model_fix.fit(X_train, y_train, epochs=20,
validation_data=(X_valid, y_valid), 
callbacks=[reduceLR])

In [ ]:
pd.DataFrame(hist_best_model_fix.history).plot(figsize=(15, 15))
plt.grid(True)
plt.gca().set_ylim(0, 2) # set the vertical range to [0-2]
plt.show()


In [ ]:
predict = best_model_fix.predict(X_test).argmax(axis=1)

In [ ]:
print(classification_report(y_test, predict))

In [ ]:
cfm = confusion_matrix(y_test, predict)
cfm_heatmap(cfm, figsize=(12,12), scale='rowwise', vmax= 0.5)